In [10]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from haversine import haversine

In [11]:
df = pd.read_csv('with_nans_in_type_and_with_random_metro.csv')
df = pd.read_csv('bhvkrsjgrvfskdf.csv')
#df = df.dropna(subset = 'Price')
#df.type = df.type.fillna('monolith')
#df['Number of rooms'].fillna(0, inplace = True)
# df = df[df.Price < 50 * 10**6]
# df = df[df.Area < 500]
df = df.drop(columns = ['Unnamed: 0',# 'id',
                        'date'])

#------------------------------------------------------------------------
### METROS
def calculate_distance(row):
    point_x = (row['lat'], row['lon'])
    point_y = (row['lat_metro'], row['lon_metro'])
    return haversine(point_x, point_y, unit='km')

list_stations = ['Ховрино', 'Сокол', 'Остров мечты', 'Минская', 'Коммунарка', 'Бульвар Генерала Карбышева', 'Удельная', 'Ольховая', 'Орехово', 'Улица Горчакова', 'Улица Академика Королёва', 'Подрезково', 'Кратово', 'Гражданская', 'Коптево', 'Пенягино', 'Железнодорожная', 'Верхние Лихоборы', 'Москва-Товарная', 'Яхромская', 'Сухаревская', 'Фрунзенская', 'Люберцы', 'Проспект Вернадского', 'Беломорская', 'Речной вокзал', 'Одинцово', 'Улица 1905 года', 'Лесной Городок', 'Марк', 'Академическая', 'Жулебино', 'Плющево', 'Силикатная', 'Троицк', 'Лубянка', 'Марьина Роща', 'Ботанический сад', 'Десна', 'Медведково', 'Новодачная', 'Полежаевская', 'Фили', 'Университет', 'Сходня', 'Цветной бульвар', 'Тимирязевская', 'Сколково', 'Крылатское', 'Комсомольская', 'Парк Победы', 'Дубровка', 'Севастопольская', 'Домодедовская', 'Тульская', 'Угрешская', 'Зорге', 'Кусково', 'Баковка', 'Серпуховская', 'Воронцовская', 'Кутузовская', 'Химки', 'Грачёвская', 'Дегунино', 'Воробьёвы горы', 'Перерва', 'Летово', 'Каспийская', 'Беляево', 'Гольяново', 'Лобня', 'Отрадное', 'Ростокино', 'Курская', 'Боровское шоссе', 'Тропарёво', 'Алма-Атинская', 'Бутово', 'Черкизовская', 'Озёрная', 'Матвеевская', 'Студенческая', 'Прокшино', 'Есенинская', 'Опалиха', 'Фонвизинская', 'Мещерская', 'Реутов', 'Бунинская аллея', 'Бульвар Рокоссовского', 'Профсоюзная', 'Шелепиха', 'Кавказский бульвар', 'Каширская', 'Лианозово', 'Суворовская', 'Перово', 'Трубная', 'Соколиная гора', 'Третьяковская', 'Новоподрезково', 'Подольск', 'Говорово', 'Аннино', 'Нахимовский проспект', 'Кузнецкий мост', 'Лухмановская', 'Верхние котлы', 'Немчиновка', 'Боровицкая', 'Улица Академика Янгеля', 'Улица Старокачаловская', 'Деловой центр', 'Хорошёвская', 'Митино', 'Рижская', 'Мякинино', 'Зеленоград — Крюково', 'Первомайская', 'Чеховская', 'Щёлковская', 'Остафьево', 'Южный порт', 'Марксистская', 'Ватутинки', 'Локомотив', 'Ильинская', 'Тютчевская', 'Кокошкино', 'Никольское', 'Терехово ', 'Тургеневская', 'Депо', 'Нагорная', 'Лихоборы', 'Народное Ополчение', 'Бибирево', 'Быково', 'Зюзино', 'Нахабино', 'Щукинская', 'Октябрьское поле', 'Липецкая', 'Останкино', 'Новоясеневская', 'Ясенево', 'Звенигородская', 'Филатов Луг', 'Петровский Парк', 'Кантемировская', 'Новокузнецкая', 'Серп и Молот', 'Волгоградский проспект', 'Шереметьевская', 'Панки', 'Долгопрудная', 'Войковская', 'Новопеределкино', 'Ипподром', 'Китай-город', 'Люблино', 'Сортировочная', 'Окружная', 'Рассказовка', 'Южная', 'Спортивная', 'Серебряный Бор', 'Рязанский проспект', 'Вешняки', 'Петровско-Разумовская', 'Нагатинская', 'Киевская', 'Динамо', 'Толстопальцево', 'Томилино', 'Площадь Ильича', 'Потапово', 'Кропоткинская', 'Тушинская', 'Вавиловская', 'Славянский бульвар', 'Левобережная', 'Очаково', 'Сетунь', 'Раменское', 'Красный Строитель', 'Мнёвники', 'Смоленская', 'Белокаменная', 'Измайлово', 'Телецентр', 'Фирсановская', 'Кузьминки', 'Октябрьская', 'Мичуринец', 'Красные ворота', 'Новаторская', 'Щербинка', 'Пушкинская', 'Балтийская', 'Битцевский парк', 'Проспект Мира', 'Сходненская', 'Кленовый бульвар', 'ВДНХ', 'Чистые пруды', 'Покровское', 'Победа', 'Менделеевская', 'Отдых', 'Измайловская', 'Партизанская', 'Лермонтовский проспект', 'Площадь Гагарина', 'Алтуфьево', 'Владыкино', 'Ольгино', 'Кучино', 'Технопарк', 'Бирюлёво', 'Физтех', 'Битца', 'Бачуринская', 'Аэропорт', 'Бульвар Дмитрия Донского', 'Университет Дружбы Народов', 'Свиблово', 'Шаболовская', 'ЗИЛ', 'Беговая', 'Павшино', 'Кожуховская', 'Молжаниново', 'Охотный ряд', 'Улица Дмитриевского', 'Площадь трёх вокзалов', 'Ухтомская', 'Москворечье', 'Шипиловская', 'Лебедянская', 'Пролетарская', 'Новогиреево', 'Сокольники', 'Панфиловская', 'Краснопресненская', 'Театральная', 'Селигерская', 'Некрасовка', 'Трикотажная', 'Чухлинка', 'Таганская', 'Достоевская', 'Дмитровская', 'Маяковская', 'Тёплый Стан', 'Солнцево', 'Апрелевка', 'Савёловская', 'Аникеевка', 'Липовая Роща', 'Новокосино', 'Авиамоторная', 'Автозаводская', 'Москва-Сити', 'Курьяново', 'Рабочий посёлок', 'Волжская', 'Котельники', 'Каховская', 'Юго-Восточная', 'Римская', 'Водники', 'Волоколамская', 'Печатники', 'Стрешнево', 'Саларьево', 'Бутырская', 'Калужская', 'Пятницкое шоссе', 'Павелецкая', 'Варшавская', 'Солнечная', 'Румянцево', 'Улица Милашенкова', 'Фабричная', 'Поклонная', 'Хлебниково', 'Улица Сергея Эйзенштейна', 'Коньково', 'Красногвардейская', 'Лужники', 'Братиславская', 'Бульвар Адмирала Ушакова', 'Карамышевская', 'Калитники', 'Корниловская', 'Площадь Революции', 'Красногорская', 'Пионерская', 'Малаховка', 'Окская', 'Зябликово', 'Новослободская', 'Парк Культуры', 'Пражская', 'Бабушкинская', 'Добрынинская', 'Сретенский бульвар', 'Крестьянская застава', 'Чертановская', 'Пыхтино', 'ЦСКА', 'Косино', 'Красносельская', 'Баррикадная', 'Санино', 'Молодёжная', 'Давыдково', 'Крёкшино', 'Арбатская', 'Улица Скобелевская', 'Нижегородская', 'Чкаловская', 'Полянка', 'Текстильщики', 'Водный стадион', 'Генерала Тюленева', 'Выставочный центр', 'Аминьевская', 'Филёвский парк', 'Хорошёво', 'Библиотека им. Ленина', 'Выхино', 'Лефортово', 'Шоссе Энтузиастов', 'Новохохловская', 'Спартак', 'Крымская', 'Тверская', 'Моссельмаш', 'Кедровая', 'Электрозаводская', 'Алексеевская', 'Бескудниково', 'Кунцевская', 'Бауманская', 'Юго-Западная', 'Аэропорт Внуково', 'Белорусская', 'Красный Балтиец', 'Переделкино', 'Ленинский проспект', 'Салтыковская', 'Борисово', 'Красково', 'Александровский сад', 'Строгино', 'Новые Черёмушки', 'Царицыно', 'Марьино', 'Мичуринский проспект', 'Нагатинский Затон', 'Багратионовская', 'Стахановская', 'Андроновка', 'Преображенская площадь', 'Лесопарковая', 'Внуково', 'Ломоносовский проспект', 'Планерная', 'Семёновская', 'Сосенки', 'Раменки', 'Коломенская']
list_stations = [station.lower().replace(' ', '') for station in list_stations]
df = df[df['Metro station'].isin(list_stations)]

# metros = pd.read_csv('metros.csv', sep = ';', header = None)
# cols = ['Metro station', 'lat_metro', 'lon_metro']
# metros.columns = cols
# metros['Metro station'] = metros['Metro station'].str.lower()
# metros['Metro station'] = metros['Metro station'].str.replace(' ', '')
# df = pd.merge(df, metros, on = 'Metro station', how = 'left')
# df['distance_km'] = df.apply(calculate_distance, axis=1)

def calculate_distance_to_red_square(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.753544, 37.621202)
    return haversine(point_x, point_y, unit='km')
df['distance_to_red_square'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_moscow_city(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.74900, 37.53964)
    return haversine(point_x, point_y, unit='km')
df['distance_to_moscow_city'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_vdnh(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.831388, 37.629277)
    return haversine(point_x, point_y, unit='km')
df['distance_to_vdnh'] = df.apply(calculate_distance_to_red_square, axis=1)

def calculate_distance_to_svo(row):
    point_x = (row['lat'], row['lon'])
    point_y = (55.968071, 37.434100)
    return haversine(point_x, point_y, unit='km')
df['distance_to_svo'] = df.apply(calculate_distance_to_red_square, axis=1)
# --------------------------------------------------------------------------


df = df.dropna(subset = 'type') 
df = df.drop(columns = ['Metro station'])
df = pd.get_dummies(df)
df

,lon,lat,Number of floors,Number of rooms,id,Area,floor,Price,distance_to_red_square,distance_to_moscow_city,...,type_block,type_brick,type_foamConcreteBlock,type_gasSilicateBlock,type_monolith,type_monolithBrick,type_old,type_panel,type_stalin,type_wood
975,37.372151,55.532642,12,0.0,301155700,23.90,5,6800000.0,29.113631,29.113631,...,False,False,False,False,True,False,False,False,False,False
1340,37.372467,55.515987,3,1.0,299151555,42.80,2,12700000.0,30.683791,30.683791,...,False,False,False,False,True,False,False,False,False,False
1727,37.320676,55.504861,14,2.0,294880080,47.10,11,15300000.0,33.474629,33.474629,...,False,False,False,False,True,False,False,False,False,False
1734,37.168891,55.587372,9,0.0,291213865,25.91,8,6214704.0,33.851409,33.851409,...,False,False,False,False,True,False,False,False,False,False
1735,37.167660,55.603110,16,4.0,270537621,132.00,4,18600000.0,32.990451,32.990451,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,37.586338,55.740517,17,2.0,300321134,45.70,10,26000000.0,2.619039,2.619039,...,False,False,False,False,False,False,False,True,False,False
52604,37.576313,55.741029,9,2.0,300324717,51.00,2,31000000.0,3.135171,3.135171,...,False,True,False,False,False,False,False,False,False,False
52605,37.576879,55.718633,16,2.0,301216303,57.00,3,39000000.0,4.771662,4.771662,...,False,True,False,False,False,False,False,False,False,False
52607,37.574489,55.739843,10,3.0,300774306,61.40,2,40000000.0,3.296732,3.296732,...,False,True,False,False,False,False,False,False,False,False


In [12]:
df = pd.get_dummies(df)#Разбиение на таргет и признаки
X = df.drop(columns=['Price'])  # Признаки 
y = df['Price']  # Таргет
#разбиение на тестовую и обучающую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [8]:
params = {
    'objective': 'regression',  # Функция потерь для регрессии
    'metric': 'rmse',  # Метрика качества
    'learning_rate': 0.125,  # Скорость обучения
    'max_depth': 7,  # Максимальная глубина деревьев
    'seed': 42  # Для воспроизводимости результатов
}

In [13]:
num_rounds = 3000  # Количество итераций (деревьев)
model = lgb.train(params, train_data, num_rounds, valid_sets=[test_data])

# Прогноз
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Оценка производительности модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с LightGBM:', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2191
[LightGBM] [Info] Number of data points in the train set: 28893, number of used features: 18
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 19038482.416295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
model = lgb.LGBMRegressor()

# Задание сетки гиперпараметров для перебора
param_grid = {
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [6, 8, 10],
    'n_estimators': [1000, 2000, 3000, 5000]
}

# Создание объекта GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

# Выполнение перебора гиперпараметров на обучающих данных
grid_search.fit(X_train, y_train)

# Получение лучших гиперпараметров
best_params = grid_search.best_params_
print("Лучшие гиперпараметры:", best_params)

# Получение лучшей модели
best_model = grid_search.best_estimator_

# Прогноз с использованием лучшей модели
y_pred = best_model.predict(X_test)

# Оценка производительности лучшей модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с LightGBM (лучшая модель):', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000791 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2184
[LightGBM] [Info] Number of data points in the train set: 19262, number of used features: 18
[LightGBM] [Info] Start training from score 19010107.193023
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV] END .learning_rate=0.05, max_depth=6, n_estimators=1000; total time=   1.3s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000630 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2186
[LightGBM] [Info] Number of data points in the train set: 19262, number of used features: 18
[LightGBM] [Info] Start training from score 19086495.468591
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [15]:
model.('model.bin')
# model = CatBoostRegressor()
# model.load_model('model.bin')

AttributeError: 'LGBMRegressor' object has no attribute 'save_model'